find cluster for original data
find cluster for permutation data

输入是从R那里得到的时空点的t p csv文档。

你算中介model的时候其实有两类
1 每个点都算
2 时间切片酸



这段代码的函数有被重用的可能性，其实应该弄到 .py 文件里，调用来使用。

这样你就不用每次跑到这个脚本里来跑代码，而是在单一的 脚本文件里 调用就行

那直接把脚本转换成

----

why use p value rather than t value to find clusters?

notice: you cannot use t-threshold to find clusters, because there is no constant t threshold for 
each temporal-spatial point.
For each temporal-spatial point, the t-threshold is different.
Because the degree of freedom is different for linear mixed model!

So you read the rds file to load summary results, and extract the p value for each temporal-spatial point.
we have to use the p value to find clusters.



calculate the permutation sum-t

prodedure:
1 calculate sum-t
'''

In [1]:
import os
import glob
import mne
import mne.stats.cluster_level_backup as cluster_level_backup
from importlib import reload
import utils_EEG.stuff as stuff
from concurrent.futures import ProcessPoolExecutor, as_completed
from mne.channels import find_ch_adjacency
import pandas as pd
import numpy as np
from datetime import datetime
# Reload custom module to ensure latest changes
reload(stuff)

reload ok
reload stuff module
D:\ProgramData\anaconda3\envs\mne12\python.exe
导入成功
reload stuff module
D:\ProgramData\anaconda3\envs\mne12\python.exe
导入成功


<module 'utils_EEG.stuff' from 'D:\\LYW\\pre10\\utils_EEG\\stuff.py'>

## define function to find clusters using sum-t method from csv file with p-values

用p还是用t，需要改两个地方
1 除了设定 threshold_type 为 'p' 或 't' 外
2 还要把 p_values 放到参数中

    threshold_type= threshold_type, # 'p' or 't'

    Extract the p-values column
    p_values = data['p_value'].values # if you want to use p-values instead of t-values
    p_values=p_values # if you want to use p-values instead of t-values
        

函数需要和调用在一个block中才能debug跳转。



In [2]:
# Modify the process_csv_file function to use sum-t clustering
# include
# find cluster 
def calculate_sumT_from_csv_file(
        csv_file_path,
        adjacency2, 
        test_iterations,
        sigma, 
        threshold_sum_t, 
        max_step, 
        t_power, 
        threshold_type,
        tail=0
):

    print(f"Processing {csv_file_path}")

    '''
    why don't you use cluster_level_backup.spatio_temporal_cluster_test() directly?
    because you didn't add a new parameter to read csv file.
    '''

    # Compute t based on the csv file
    # using coef_estimate and std_error from csv file
    # and sigma to smooth the t values.
    # I should change the function name to calculate_t_values_from_csv_file。。。
    # smoothing..
    # if you don't need smoothing, then set sigma = 0 or load from csv file directly
    t_obs_orig_permu = stuff.apply_lmer_models_to_array_parallel_hat(
        array_of_dfs=None,
        test_iterations=test_iterations,#useless
        max_workers=5,# useless,no parallel process in this version function.
        sigma=sigma,
        csv_file_path=csv_file_path # default csv file path: summary_results_2024年11月14日.csv
    )

    # load p-values from csv file
    data = pd.read_csv(csv_file_path)
    
    # Extract the p-values column
    p_values = data['p_value'].values # if you want to use p-values instead of t-values

    # Calculate max and min values of t_obs_orig_permu
    max_t_obs = np.max(t_obs_orig_permu)
    min_t_obs = np.min(t_obs_orig_permu)
    print(f"Max value of t_obs_orig_permu: {max_t_obs}")
    print(f"Min value of t_obs_orig_permu: {min_t_obs}")

    # Find clusters using sum-t method
    # I have to change the function to use p-values instead of t-values for the clustering.
    out = cluster_level_backup._find_clusters_TorP(
        x=t_obs_orig_permu,
        threshold=threshold_sum_t,
        tail=tail,
        adjacency=adjacency2,
        max_step=max_step,
        include=None,
        partitions=None,
        t_power=t_power,
        show_info=True,
        # additional
        threshold_type= threshold_type, # 'p' or 't'
        p_values=p_values # if you want to use p-values instead of t-values
    )
    clustersT, clusterT_sum = out

    return csv_file_path, {'out': out} # if you want all cluster information, you can return it.
    # # Calculate min and max of cluster sums
    # min_cluster_sum = np.min(clusterT_sum)
    # max_cluster_sum = np.max(clusterT_sum)
    #
    # return csv_file_path, min_cluster_sum, max_cluster_sum


# Define the set of channels to use (excluding FT9, FT10, TP9, TP10)
channels_all = [
    'Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
    'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8',
    'Fz', 'Cz', 'Pz', 'FC1', 'FC2', 'CP1', 'CP2',
    'FC5', 'FC6', 'CP5', 'CP6', 'F1', 'F2', 'C1',
    'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4',
    'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5',
    'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8',
    'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'CPz', 'POz', 'Oz'
]

# Load an evoked file as an example
file_path = r"D:\LYW\pre10\data\7evoked_allWords\prex006-ave.fif"
evoked_example = mne.read_evokeds(file_path, proj=False, verbose=None)
evoked_example = evoked_example[0].resample(91)  # Resample to 91 Hz
evoked_example.pick_channels(channels_all)  # Select a subset of channels

# Compute adjacency for the chosen channels
info = evoked_example.info
adjacency, ch_names = find_ch_adjacency(info, ch_type="eeg")

# Set parameters related to cluster analysis
max_step = 1
t_power = 1
n_tests = 59 * 91
n_times = 91

# Setup adjacency structure required for TFCE calculations
adjacency2 = cluster_level_backup._setup_adjacency(adjacency, n_tests, n_times)

test_iterations = 5733 # useless


Reading D:\LYW\pre10\data\7evoked_allWords\prex006-ave.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms (0.40 × 100 + 0.20 × 101 + 0.20 × 102 + 0.20 × 103)
        0 CTF compensation matrices available
        nave = 250 - aspect type = 100
Loaded Evoked data is baseline-corrected (baseline: [-0.2, 0] s)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 59


## 找cluster 

bug: 为何算出来的sum_T 都是inf?

用t或者用p要改两个地方：
threshold_sum_t = 0.01 # if you set threshold_type='p',
threshold_sum_t = 1.96 # if you set threshold_type='t',
    'p' # if you set threshold_type='p',
    't' # if you set threshold_type='t',


无论是对多个permutation的结果找cluster，还是对原始数据结果找cluster，都调用一样的函数的。

而t值根据各种情况不同而不同。
1 一开始的时候是根据mix effect model算的t，亮哥教你的算法。
2 你后来做的用mplus，各种estimator算的，中介模型。
    a b ab



D:\LYW\pre10\utils_EEG\拿到a b ab的z p.ipynb
算出来：
output_file = f"D:\\LYW\\pre10\\data\\permutation_summary_between\\summary_results_2025年5月28日_ab_updated.csv"

### d

In [19]:


'''
D:\\LYW\R for permutation t\\Mplus\\results\\all_spatiotemporal_results_WLSMV_precise_parallel.csv
这个文件地址算的结果包含了 a b c ab 的精确 estimate 和  std_error
这样就可以判断 estimate的正负，函数才能起作用。

直接把 ab_estimate_dat	ab_std_error_dat改成 coef_estimate std_error 就行了，然后相除得到另一行 t_value


需要注意，你关心的是 a为负，但是间接效应ab是正的。
所以 对于ab,你需要设置 tail=1，t = 1.96

ab>0,然后要分两类
a<0  分散学习带来更多变异，那么因该出现在中晚期，促进逻辑分上升。
a>0  集中学习带来更多变异，出现在早期，这种变异遏制逻辑分上升。
一开始的方向有正负 后面对应的是 促进 和 遏制，负负得正 才是一对变化。

那还得再改一次Excel表格
'''

'''

需要对原始的 csv文件做一些预处理

1 允许我去选择 a b ab，增加四列coef_estimate std_erorr ，t_value p_value, 就根据其中对应的两列数据来得到！
2  如果是ab，那么需要通过a 大于0 小于0 来分成两个文档。


'''
def add_standard_stats_columns(csv_file_path, effect_type='ab',):
      """
      Add standard statistical columns (coef_estimate, std_error,
  t_value, p_value)
      based on selected effect type (a, b, or ab).

      Parameters:
      - csv_file_path: str, path to the CSV file containing mediation
  results
      - effect_type: str, 'a', 'b', or 'ab' to select which effect to
  analyze
      - output_path: str, optional path to save the modified CSV. If
  None, returns DataFrame only.

      Returns:
      - pandas.DataFrame with added columns
      """
      import pandas as pd
      import numpy as np
      from scipy import stats

      # Load data
      data = pd.read_csv(csv_file_path)

      # Define column mappings based on effect type
      column_mapping = {
          'a': {'estimate': 'a_estimate_dat', 'std_error':
  'a_std_error_dat'},
          'b': {'estimate': 'b_estimate_dat', 'std_error':
  'b_std_error_dat'},
          'ab': {'estimate': 'ab_estimate_dat', 'std_error':
  'ab_std_error_dat'}
      }

      if effect_type not in column_mapping:
          raise ValueError("effect_type must be 'a', 'b', or 'ab'")

      # Get the appropriate columns
      estimate_col = column_mapping[effect_type]['estimate']
      std_error_col = column_mapping[effect_type]['std_error']

      # Add standard columns
      data['coef_estimate'] = data[estimate_col]
      data['std_error'] = data[std_error_col]

      # Calculate t-values (avoid division by zero)
      with np.errstate(divide='ignore', invalid='ignore'):
          data['t_value'] = data['coef_estimate'] / data['std_error']
          data['t_value'] = data['t_value'].replace([np.inf, -np.inf],
  np.nan)

      # Calculate p-values (two-tailed test)
      data['p_value'] = 2 * (1 -
  stats.norm.cdf(np.abs(data['t_value'])))
      data['p_value'] = data['p_value'].fillna(1.0)  # NaN t-values get p=1

      print(f"Added standard columns based on {effect_type} effect:")
      print(f"  coef_estimate <- {estimate_col}")
      print(f"  std_error <- {std_error_col}")
      print(f"  t_value = coef_estimate / std_error")
      print(f"  p_value = 2 * (1 - norm.cdf(|t_value|))")

    # Generate output path by adding effect_type to original filename
      base_name, ext = os.path.splitext(csv_file_path)
      output_path = f"{base_name}_{effect_type}{ext}"

      # Save the modified data
      data.to_csv(output_path, index=False)
      print(f"Saved to: {output_path}")

      return data

 # 增加_dat 高精度数据
csv_path = r"D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel.csv"
df_ab = add_standard_stats_columns(csv_path, effect_type='ab')
df_a = add_standard_stats_columns(csv_path, effect_type='a')
df_b = add_standard_stats_columns( csv_path, effect_type='b')

Added standard columns based on ab effect:
  coef_estimate <- ab_estimate_dat
  std_error <- ab_std_error_dat
  t_value = coef_estimate / std_error
  p_value = 2 * (1 - norm.cdf(|t_value|))
Saved to: D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_ab.csv
Added standard columns based on a effect:
  coef_estimate <- a_estimate_dat
  std_error <- a_std_error_dat
  t_value = coef_estimate / std_error
  p_value = 2 * (1 - norm.cdf(|t_value|))
Saved to: D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_a.csv
Added standard columns based on b effect:
  coef_estimate <- b_estimate_dat
  std_error <- b_std_error_dat
  t_value = coef_estimate / std_error
  p_value = 2 * (1 - norm.cdf(|t_value|))
Saved to: D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_b.csv


In [18]:
'''

如果是ab，那么需要通过a 大于0 小于0 来分成两个文档。

我现在需要把 "D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_ab.csv" 这个文件变成两个。
其中一个，要求检查每一行 a_estimate_dat，如果大于0，那么t_value 被赋值为0 ，保存all_spatiotemporal_results_WLSMV_precise_parallel_ab_a大于0
另一个反过来。


这个脚本会：

  1. 读取原始CSV文件
  2. 添加标准统计列 (基于ab效应)
  3. 创建两个副本：
    - all_spatiotemporal_results_WLSMV_precise_parallel_ab_a小于等于0.csv:
  当 a_estimate_dat > 0 时，将 t_value 设为0 (保留a<=0的效应)
    -
  all_spatiotemporal_results_WLSMV_precise_parallel_ab_a大于0.csv:
  当 a_estimate_dat <= 0 时，将 t_value 设为0 (保留a>0的效应)
  4. 同时调整p值: 当t_value被设为0时，p_value也设为1.0

----

a 和 b 就不需要split了。

'''
import pandas as pd
import numpy as np
from scipy import stats
import os

def split_csv_by_a_estimate(csv_file_path):
  """
  Split CSV file into two based on a_estimate_dat values, with
conditional t_value assignment.

  Parameters:
  - csv_file_path: str, path to the original CSV file
  """

  # Load data
  data = pd.read_csv(csv_file_path)

  # Add standard statistical columns for ab effect
  data['coef_estimate'] = data['ab_estimate_dat']
  data['std_error'] = data['ab_std_error_dat']

  # Calculate t-values
  with np.errstate(divide='ignore', invalid='ignore'):
      data['t_value'] = data['coef_estimate'] / data['std_error']
      data['t_value'] = data['t_value'].replace([np.inf, -np.inf],
np.nan)

  # Calculate p-values
  data['p_value'] = 2 * (1 -
stats.norm.cdf(np.abs(data['t_value'])))
  data['p_value'] = data['p_value'].fillna(1.0)

  # Create two copies of the data
  data_when_positive_zero = data.copy()  # a>0时t_value=0，保留a<=0的效应
  data_when_negative_zero = data.copy()  # a<=0时t_value=0，保留a>0的效应

  # When a_estimate > 0, set t_value to 0 (keeping only negative a effects)
  mask_positive = data_when_positive_zero['a_estimate_dat'] >= 0
  data_when_positive_zero.loc[mask_positive, 't_value'] = 0
  data_when_positive_zero.loc[mask_positive, 'coef_estimate'] = 0  # 同时将coef_estimate设为0
  data_when_positive_zero.loc[mask_positive, 'p_value'] = 1.0  # p-value = 1   when t = 0

  # When a_estimate <= 0, set t_value to 0 (keeping only positive a effects)
  mask_negative = data_when_negative_zero['a_estimate_dat'] <= 0
  data_when_negative_zero.loc[mask_negative, 't_value'] = 0
  data_when_negative_zero.loc[mask_negative, 'coef_estimate'] = 0  # 同时将coef_estimate设为0
  data_when_negative_zero.loc[mask_negative, 'p_value'] = 1.0  # p-value = 1     when t = 0

  # Generate output paths with corrected names
  base_name, ext = os.path.splitext(csv_file_path)
  output_path_keep_negative = f"{base_name}_a小于等于0{ext}"  # a>0时被置零，保留a<=0
  output_path_keep_positive = f"{base_name}_a大于0{ext}"     # a<=0时被置零，保留a>0

  # Save the files
  data_when_positive_zero.to_csv(output_path_keep_negative, index=False)
  data_when_negative_zero.to_csv(output_path_keep_positive, index=False)

  print(f"文件已分割:")
  print(f"保留 a_estimate <= 0 的效应: {output_path_keep_negative}")
  print(f"  - 有 {mask_positive.sum()} 行 (a>0) 被设为 t_value=0")
  print(f"保留 a_estimate > 0 的效应: {output_path_keep_positive}")
  print(f"  - 有 {mask_negative.sum()} 行 (a<=0) 被设为 t_value=0")

  return data_when_positive_zero, data_when_negative_zero

# 使用示例
csv_path = r"D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_ab.csv"
df_keep_neg, df_keep_pos = split_csv_by_a_estimate(csv_path)

文件已分割:
保留 a_estimate <= 0 的效应: D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_ab_a小于等于0.csv
  - 有 1988 行 (a>0) 被设为 t_value=0
保留 a_estimate > 0 的效应: D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_ab_a大于0.csv
  - 有 3381 行 (a<=0) 被设为 t_value=0


### 算

In [23]:
# 这两个是ab的
csv_files = ["D:\\LYW\R for permutation t\\Mplus\\results\\all_spatiotemporal_results_WLSMV_precise_parallel_ab_a小于等于0.csv"]
# csv_files = ["D:\\LYW\R for permutation t\\Mplus\\results\\all_spatiotemporal_results_WLSMV_precise_parallel_ab_a大于0.csv"]

# # a 只要一个就行了，修改对应的 t  和 tail 的正负号就行了。
# csv_files = ["D:\\LYW\R for permutation t\\Mplus\\results\\all_spatiotemporal_results_WLSMV_precise_parallel_a.csv"]
#
# # b 也一样吧
# csv_files = ["D:\\LYW\R for permutation t\\Mplus\\results\\all_spatiotemporal_results_WLSMV_precise_parallel_b.csv"]



#############################################################################
# original value file
# summary_results_2024年11月14日_upaded.csv 这个文件是从D:/LYW/R for permutation t/loadRDS.qmd 中复制出来的，
# 这个是mix effect model 算的结果，其实相当于后面算的 a_path
# csv_files = ["D:\LYW\pre10\data\permutation_summary_between\summary_results_2024年11月14日_upaded.csv"]
# 这个是后面用中介模型算的结果， ab代表的是ab_path 算出来的显著点。
# 筛选条件：ab显著为正，a显著为负，b显著为负
# 问题：哪里的代码进行了筛选，难道是你之前手动筛选的？
# 2025年7月22日到这个日期为止你是自己进行了筛选，把不符合要求的p改成了1 ，实际上是不对的？
# csv_files = ["D:\\LYW\\pre10\\data\\permutation_summary_between\\summary_results_2025年05月28日_ab_updated.csv"]
# 中介 - a
# csv_files = ["D:\\LYW\\pre10\\data\\permutation_summary_between\\summary_results_2025年05月29日_a_updated.csv"]
#
# # 混合效应模型,这个适用的model formula <- 'distance ~ category_cond2 + logicalScore1Diff + RT1Diff + (1|subId_cond1) ' 差值。
# # 这里没有p值，因为你的R代码注释掉了生成RDS文件。
# csv_files = ["D:\\LYW\\pre10\\data\8 temporal-spatial\\sub28\\summary_coef_std_20250704_133507.csv"]


In [24]:
# 这个csv文件可以从别的地方来。
# 
# Define a fixed threshold for sum-t clustering
 # Example threshold value, not like tfce, we have to set it manually.
 # not threshold free!
# threshold_sum_t = 0.05 # if you set threshold_type='p',

# 正负也要和tail一起变化?
threshold_sum_t = 1.96 # if you set threshold_type='t', 1.96对应的是0.05, 2.576对应0.01,0.001 对应3.291

# Tail parameter (set here to 1, meaning similarity massed < spaced, dissimilarity spaced < massed)
# 这个tail是全局变量，所以是有用的。
# 如果算的是ab，那么就肯定是正
tail = 1

# sigma = 0.001  # Set the smoothing parameter
sigma = 0.01  # Set the smoothing parameter
# sigma = 0 # I don't need smoothing for sum-t clustering

# For debugging
print(f"adjacency2: {type(adjacency2)}")
print(f"n_tests: {type(n_tests)} - {n_tests}")
print(f"n_times: {type(n_times)} - {n_times}")
print(f"threshold_tfce: {type(threshold_sum_t)} - {threshold_sum_t}")

# one shot test
#
csv_file_path, out = calculate_sumT_from_csv_file(
    csv_files[0],
    adjacency2,
    test_iterations,
    sigma,
    threshold_sum_t,
    max_step,
    t_power,
    # 'p' ,# if you set threshold_type='p',
    't' ,# if you set threshold_type='t',
    tail = tail
)
# Assuming out is a tuple (clustersT, clusterT_sum)
clustersT, clusterT_sum = out['out']

'''
storing as csv file cannot keep the structure of the data.
'''
# # Convert to DataFrame
# df = pd.DataFrame({
#     'Cluster Indices': clustersT,
#     'Cluster Sums': clusterT_sum
# })
# 
# # Get the current date
current_date = datetime.now().strftime("%Y%m%d")
# 
# # Specify the file path with date and threshold information
# output_csv_path = f"D:\\LYW\\pre10\\data\\permutation_summary_between\\summary_results_{current_date}_thresh_{threshold_sum_t}_clustersT.csv"
# 
# 
# # Write the DataFrame to a CSV file
# df.to_csv(output_csv_path, index=False)
# 
# print(output_csv_path)

import pickle
# 
# # Store clustersT using pickle
# pickle_path = f"D:\\LYW\\pre10\\data\\permutation_summary_between\\clustersT_{current_date}_thresh_{threshold_sum_t}.pkl"
# with open(pickle_path, 'wb') as f:
#     pickle.dump(clustersT, f)
# 
# print(f"Clusters stored at: {pickle_path}")
# 
# # Reload clustersT from the pickle file
# with open(pickle_path, 'rb') as f:
#     clustersT = pickle.load(f)
# 
# # Verify the data
# print(clustersT)
# 

# 自动从csv文件名中提取信息来构建pickle文件名
csv_filename = os.path.basename(csv_files[0])  # 获取文件名，例如: all_spatiotemporal_results_WLSMV_precise_parallel_ab_a大于0.csv
csv_name_without_ext = os.path.splitext(csv_filename)[0]  # 去掉.csv扩展名

# Store clustersT and clusterT_sum using pickle with auto-generated name
pickle_path = f"D:\\LYW\\pre10\\data\\permutation_summary_between\\clusters_{current_date}_thresh_{threshold_sum_t}_direction_{tail}_{csv_name_without_ext}.pkl"
with open(pickle_path, 'wb') as f:
    pickle.dump((clustersT, clusterT_sum), f)

print(f"Clusters stored at: {pickle_path}")

# Reload clustersT and clusterT_sum from the pickle file
with open(pickle_path, 'rb') as f:
    clustersT2, clusterT_sum2 = pickle.load(f)

# Verify the data
print(clustersT2)
print(clusterT_sum2)


# 注意看找到的cluster保存在哪个文件里了Clusters stored at: D:\LYW\pre10\data\permutation_summary_between\clusters_20250704_thresh_0.01.pkl
# 到这里可以回到D:\LYW\pre10\analysis\nps\nps_dist_mix_sep_epoch_searchlight_2025年1月2日.ipynb ： find clusters - sum_t version - considering different dgree of freedom

adjacency2: <class 'list'>
n_tests: <class 'int'> - 5369
n_times: <class 'int'> - 91
threshold_tfce: <class 'float'> - 1.96
Processing D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_ab_a小于等于0.csv
***Loading t values from CSV***, set globally before apply_lmer_models_to_array_parallel_hat
D:\LYW\R for permutation t\Mplus\results\all_spatiotemporal_results_WLSMV_precise_parallel_ab_a小于等于0.csv
Max std_error (ignoring NaNs): 0.00049637841
Results saved to results_t_original.npy and results_t_smooth.npy
Max value of t_obs_orig_permu: 10.361620506609512
Min value of t_obs_orig_permu: -6.6462751355121465
Clusters stored at: D:\LYW\pre10\data\permutation_summary_between\clusters_20250904_thresh_1.96_direction_1_all_spatiotemporal_results_WLSMV_precise_parallel_ab_a小于等于0.pkl
[array([1337, 1365, 1396, 1410, 1414, 1424, 1455, 1469, 1430, 1461, 1483,
       1514, 1528, 1489, 1520, 1526, 1362, 1421, 1446, 1480, 1505, 1492,
       1539, 1564, 1578, 1551, 1

## calculate the min and max cluster sums for each permutation - only save the max cluster sums

和算原始数据的cluster不一样，并不是保存所有的cluster，每个permutation，只需要保存最大的那个就行。

In [4]:
import os
import glob
import pandas as pd

# Directory containing the CSV files for permutation summary
csv_dir = r"H:\\lyw\\sub28"
csv_files = glob.glob(os.path.join(csv_dir, "summary_results_permutation_[0-9]*.csv"))
# Filter out anything that contains "_updated"
csv_files = [f for f in csv_files 
             if "_updated" not in os.path.basename(f) 
             and "updated_clustersT" not in os.path.basename(f)]

In [ ]:
# Parameters
'''
p-value	One-tailed Z-value	Two-tailed Z-value
0.05	1.645	±1.96
0.01	2.33	±2.58
'''

threshold_sum_t = 0.01 # Example threshold for p-value
threshold_sum_t = 2.58 # Example threshold for t-value
tail = 0
sigma = 0.001
max_step = 1  # Example value, adjust as needed
t_power = 1  # Example value, adjust as needed

# List to store results for each file
cluster_sums_stats = []

file_limit = len(csv_files)  # Limit the number of files to process
# file_limit = 2
# Process each CSV file
for i, csv_file in enumerate(csv_files[:file_limit]):

    # Perform the calculation
    out = calculate_sumT_from_csv_file(
        csv_file,
        adjacency2,
        test_iterations,
        sigma,
        threshold_sum_t,
        max_step,
        t_power,
        threshold_type='t'
    )
    
    # Assuming out is a tuple (clustersT, clusterT_sum)
    clustersT, clusterT_sum = out
    
    # Calculate min and max from the second element of clusterT_sum['out'] 
    # which is the NumPy array
    # min_sum = clusterT_sum['out'][1].min()
    # max_sum = clusterT_sum['out'][1].max()
    # Check if 'out' key exists and has enough elements
    if 'out' in clusterT_sum and len(clusterT_sum['out']) > 1:
        cluster_array = clusterT_sum['out'][1]
        if cluster_array.size > 0:
            min_sum = cluster_array.min()
            max_sum = cluster_array.max()
        else:
            min_sum = None
            max_sum = None
    else:
        min_sum = None
        max_sum = None
    
    print(f"For {csv_file}: min cluster sum = {min_sum}, max cluster sum = {max_sum}")

    # Append the results as a dictionary
    cluster_sums_stats.append({
        'File': csv_file,
        'MinClusterSum': min_sum,
        'MaxClusterSum': max_sum
    })
    
    print(f"Processed {csv_file} and saved stats.")

# Convert the list of dicts to a DataFrame
cluster_sums_df = pd.DataFrame(cluster_sums_stats)

# Get the current date
current_date = datetime.now().strftime("%Y%m%d")

# Specify the file path with date and threshold information
output_csv_path = os.path.join(
    csv_dir,
    f"min_max_cluster_sums_{current_date}_thresh_{threshold_sum_t}.csv"
)

# Write the DataFrame to a CSV file
cluster_sums_df.to_csv(output_csv_path, index=False)

print(f"Saved min and max cluster sums to {output_csv_path}")


In [ ]:
# test load csv file
# Load the CSV file into a DataFrame
cluster_sums_df = pd.read_csv(output_csv_path)
# Verify the data
print(cluster_sums_df.head())


## veiw the min and max cluster sums distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Drop rows with NaN values
cluster_sums_df = cluster_sums_df.dropna(subset=['MinClusterSum', 'MaxClusterSum'])


# Assume you have a DataFrame named cluster_sums_df
# with columns 'MinClusterSum' and 'MaxClusterSum'

# Create a figure with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

# Plot histogram of MinClusterSum
sns.histplot(data=cluster_sums_df, x='MinClusterSum', kde=True, ax=axs[0], color='blue')
axs[0].set_title('Distribution of Min Cluster Sum')

# Plot histogram of MaxClusterSum
sns.histplot(data=cluster_sums_df, x='MaxClusterSum', kde=True, ax=axs[1], color='red')
axs[1].set_title('Distribution of Max Cluster Sum')

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

alpha = 0.01
lower_pct = 100 * (alpha / 2)      # 2.5 percentile
upper_pct = 100 * (1 - alpha / 2)  # 97.5 percentile

# Calculate thresholds for MinClusterSum
min_lower_thresh = np.percentile(cluster_sums_df['MinClusterSum'], lower_pct)
min_upper_thresh = np.percentile(cluster_sums_df['MinClusterSum'], upper_pct)

# Calculate thresholds for MaxClusterSum
max_lower_thresh = np.percentile(cluster_sums_df['MaxClusterSum'], lower_pct)
max_upper_thresh = np.percentile(cluster_sums_df['MaxClusterSum'], upper_pct)

print(f"MinClusterSum thresholds (p=0.05, two-sided): {min_lower_thresh} to {min_upper_thresh}")
print(f"MaxClusterSum thresholds (p=0.05, two-sided): {max_lower_thresh} to {max_upper_thresh}")


In [ ]:
'''
learning 

how df influence the t distribution?
'''

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t

# Degrees of freedom to compare
dfs = [60000, 70000]

# Create an x-axis from -5 to 5
x = np.linspace(-5, 5, 400)

plt.figure(figsize=(8, 6))

for df in dfs:
    # Get the t-distribution PDF for this df
    pdf = t.pdf(x, df)
    plt.plot(x, pdf, label=f"df={df}")

# Also include the standard normal distribution for comparison
from scipy.stats import norm
plt.plot(x, norm.pdf(x), label="Normal (df=∞)", linestyle='--', color='black')

plt.title("T-Distribution with Different Degrees of Freedom")
plt.xlabel("Value")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
'''
instant code 

find files' name and delete files who have been updated.\

'''

import os

directory = r'H:\lyw\sub28'

# Print only file names
for root, dirs, files in os.walk(directory):
    for file in files:
        print(file)


In [ ]:
import os
import re

# Define the directory path
directory = r'H:\lyw\sub28'

# Initialize a list to store numbers
numbers = []

# Walk through the directory
for root, dirs, files in os.walk(directory):
    for file in files:
        # Check if the file matches the pattern
        if re.match(r'summary_results_permutation_\d+\.csv', file):
            # Extract the number using regex
            num = re.search(r'\d+', file).group()
            numbers.append(int(num))

# Print the numbers
print("Extracted numbers:", numbers)


In [ ]:
# Define the full range of numbers from 1 to 1000
full_range = set(range(1, 1001))

# Example extracted numbers (as a list)
extracted_numbers = numbers

# Convert the list to a set for set operations
extracted_numbers_set = set(extracted_numbers)

# Find missing numbers
missing_numbers = sorted(full_range - extracted_numbers_set)

# Display the missing numbers
print("Missing numbers:", missing_numbers)

